In [4]:
print('juju time')

juju time


In [12]:
!python models/nonlinear_regression_models/LWPR_quantile_wind.py

CUDA device is available. Using CUDA.
Fitting points (quantiles): 100%|█████████████| 100/100 [26:18<00:00, 15.78s/it]
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Hel

In [8]:
!python models/double_machine_learning/DML_overall_wind.py

Window 1/17:   0%|                                      | 0/100 [00:00<?, ?it/s]^C
Traceback (most recent call last):                                              
  File "/home/RDC/inceemir/power/models/double_machine_learning/DML_overall_wind.py", line 45, in <module>
    df_residualized = residualize_data(random_subset, y='electricity_price')
  File "/home/RDC/inceemir/power/models/double_machine_learning/DML_utils_wind.py", line 46, in residualize_data
    data['price_resid'] = residualize_variable_kfold(
  File "/home/RDC/inceemir/power/models/double_machine_learning/DML_utils_wind.py", line 30, in residualize_variable_kfold
    model = lgb.train(params, lgb_train, num_boost_round=100)
  File "/home/RDC/inceemir/power/.venv/lib64/python3.9/site-packages/lightgbm/engine.py", line 322, in train
    booster.update(fobj=fobj)
  File "/home/RDC/inceemir/power/.venv/lib64/python3.9/site-packages/lightgbm/basic.py", line 4155, in update
    _LIB.LGBM_BoosterUpdateOneIter(
KeyboardInterru